# Optimizing Code with JuMP

Many linear algebra operations, like matrix-matrix multiply, can be written fairly simply as nested loops. However if this was literally translated into machine code then the performance you would obtain would be a fraction of what is possible. The reason is that modern CPUs are complex things, capable of running multiple instructions at a time with deep "pipelines" to take account of parallelism on a small scale.

In this notebook we will develop a model to determine the optimal scheduling for a sequence of instructions so that the CPU can pipeline the operations to perform a calculation as fast as possible.

## Four-element dot product

Consider a function that calculates the dot products of two four-element vectors. We will specify the **directed acyclic graph (DAG)** of operations that need to be performed, and then we will optimize the scheduling of the operations. For this function we will use the DAG

```
[-----]    [-----]     
[  A  ]    [  E  ]      MUL  A E   --\
[-----]    [-----]                    ADD  --\
[  B  ]    [  F  ]      MUL  B F   --/        \
[-----] .* [-----]  =                          ADD
[  C  ]    [  G  ]      MUL  C G   --\        /
[-----]    [-----]                    ADD  --/
[  D  ]    [  H  ]      MUL  D H   --/
[-----]    [-----] 
```

From this we can see that, on a very "parallel" machine, all the MULs can be performed in parallel. Two of the ADDs can then be done together, but the final ADD can only run after everything else is done. The length of time for each operation can vary. For example, MUL might take 2 units of time, while ADD only takes 1.

On a real machine we have some constraints. For this problem we will consider a CPU with the following restrictions and properties:
* Only one operation can start at a time
* ADD and MUL both take three units of time to run
* Two ADDs and two MULs can be done at time

## Modeling as an binary optimization problem

We can pose this scheduling problem as a binary linear optimization problem. We will first fix a maximum time $T$, and create a binary variable for each of the operations $i$ and times $t$. If this variable has value 1 then we say that the operation starts at time $t$

In [ ]:
# Create our list of operations
ops = [:MUL_AE, :MUL_BF, :MUL_CG, :MUL_DH, :ADD_ABEF, :ADD_CDGH, :ADD_ALL]
# Max possible finishing time
T = 20
# Length of an instruction
L = 3
# Max instructions running at a time
C = 2

# Lets optimize!
using JuMP

#using Gurobi
#m = Model(solver=GurobiSolver())
m = Model()

@defVar(m, startat[op=ops, t=1:T], Bin);

We can immediately add some constraints on `startat` that have nothing to do with the dependency graph.

In [ ]:
# Each operation should start once, and once only
for op in ops
    @addConstraint(m, sum{startat[op,t], t=1:T} == 1)
end

# At most one operation can start at each time
for t in 1:T
    @addConstraint(m, sum{startat[op,t], op=ops} <= 1)
end

# There will be multiple possible solutions, but lets
# pin it down by fixing that the first MUL starts at
# time 1
@addConstraint(m, startat[:MUL_AE,1] == 1);

Implementing precedence constraints is a little more tricky. Lets consider `ADD_ABEF`, which can only start after `MUL_AE` and `MUL_BF` have completed. Say we start `MUL_AE` at time 1, which will be done by time 4, and we start `MUL_BF` at time 2, which will be done by time 5. We can thus start the add at time 5 or later, or from another perspective, we cannot start the add unless both of the MULs were started at least 3 time units beforehand. We capture the "at least" by taking a sum over `start` for those operations across time, and enforce the relationship with a less-than-or-equal-to constraint.

In [ ]:
# For each possible starting time for the dependent operations
for t in 1:T-L
    # ADD_ABEF cannot start until MUL_AE and MUL_BF finish
    @addConstraint(m, startat[:ADD_ABEF,t] <= sum{startat[:MUL_AE,s], s=1:t-L})
    @addConstraint(m, startat[:ADD_ABEF,t] <= sum{startat[:MUL_BF,s], s=1:t-L})
    # ADD_CDGH cannot start until MUL_CG and MUL_DH finish
    @addConstraint(m, startat[:ADD_CDGH,t] <= sum{startat[:MUL_CG,s], s=1:t-L})
    @addConstraint(m, startat[:ADD_CDGH,t] <= sum{startat[:MUL_DH,s], s=1:t-L})
    # ADD_ALL cannot start until ADD_ABEF and ADD_CDGH finish
    @addConstraint(m, startat[:ADD_ALL,t] <= sum{startat[:ADD_ABEF,s], s=1:t-L})
    @addConstraint(m, startat[:ADD_ALL,t] <= sum{startat[:ADD_CDGH,s], s=1:t-L})
end

We'll try to start the last operation as soon as possible. We can do that by creating a penalty proportional to the start time, i.e. the start time itself. Then lets try solving the model with what we've got!

In [ ]:
@setObjective(m, Min, sum{t*startat[:ADD_ALL,t],t=1:T})

status = solve(m)
@show status
function pretty(op)
    println(op)
    for t in 1:T
        if iround(getValue(startat[op,t])) == 1
            print("S")
        else
            print("_")
        end
    end
    println()
end
map(pretty, ops);

We can see that the dependency is being respected, but we haven't modeled the other important constraint that we cannot run more than 2 MUL and 2 ADD at a time. Because of our problem we don't need to worry about the ADD as that never occurs, but we'll model it anyway for completeness.

There are a few approaches we could use, but we'll address this by introducing a new variable `running` that will be 1 if an operation is still running at time $t$. To do so we'll need to link `start` and `running`, but the constraint limiting the number of things run is simple.

In [ ]:
@defVar(m, running[op=ops,t=1:T], Bin)

# Link running to startat
for op in ops
    # For each possible start time
    for t in 1:T-L+1
        # For the L units of time including that start time
        for s in t:t+L-1
            @addConstraint(m, running[op,s] >= startat[op,t])
        end
    end
end

# Can't do more than a certain number of ops at a time
mul_ops = ops[1:4]
add_ops = ops[5:7]
for t in 1:T
    # Multiplies
    @addConstraint(m, sum{running[op,t],op=mul_ops} <= C)
    # Adds
    @addConstraint(m, sum{running[op,t],op=add_ops} <= C)
end

In [ ]:
status = solve(m)
@show status
function pretty2(op)
    print(rpad(op,10," "))
    for t in 1:T
        if iround(getValue(startat[op,t])) == 1
            print("S")
        elseif iround(getValue(running[op,t])) == 1
            print("R")
        else
            print("_")
        end
    end
    println()
end
map(pretty2, ops);

Depending on your solver, you may get different solutions. When I tried earlier, I got
```
MUL_AE    SRR___________________________
MUL_BF    _SRR__________________________
MUL_CG    ____SRR_______________________
MUL_DH    ___SRR________________________
ADD_ABEF  ____RRSRR_____________________
ADD_CDGH  ____RRRSRR____________________
ADD_ALL   __________SRR_________________
```
Which is odd, because there are some `R` before the `S`. Have no fear: its simply because there is no reason not to set it 1, as we are not constrained at that time, and because solvers are very literal like that. We could fix that by either adding more constraints, or adding a small objective term to discourage setting `running` to 1 if it isn't needed:

In [ ]:
@setObjective(m, Min, sum{t*startat[:ADD_ALL,t],t=1:T} + 1e-5*sum(running))
status = solve(m)
@show status
map(pretty2, ops);

## Extension: Matrix-Vector Product

In [ ]:
# We are building kernel for 4x4 matrix * vector product, i.e.
# [a11 a12 a13 a14] [x1]
# [a21 a22 a23 a24] [x2] = A x
# [a31 a32 a33 a34] [x3]
# [a41 a42 a43 a44] [x4]
# First, define an operation primitive
immutable Operation
    name::String  # User-friendly name for the operation
    optype::Symbol  # Type of operation (:ADD,:MUL,:NOOP)
    # Operations that must complete before this operation starts
    depends::Vector  
end

# Each element of the result vector is a dot product
# between a row of A and the vector x
function make_dot_product(prefix)
    Operation(prefix*"ADDFINAL", :ADD, [
        Operation(prefix*"ADD1", :ADD, [
            Operation(prefix*"MUL1", :MUL, []),
            Operation(prefix*"MUL2", :MUL, [])
        ]),
        Operation(prefix*"ADD2", :ADD, [
            Operation(prefix*"MUL3", :MUL, []),
            Operation(prefix*"MUL4", :MUL, [])
        ])
    ])
end
final = Operation("MATVEC",:NOOP,[
#    make_dot_product("ROW1"),
#    make_dot_product("ROW2"),
    make_dot_product("ROW3"),
    make_dot_product("ROW4")
])

# Collect all the operations into a vector so we
# can index over them easier
ops = Operation[]
function collect_ops(op)
    push!(ops, op)
    map(collect_ops, op.depends)
end
collect_ops(final)
@show length(ops)

# Build model
using JuMP
#using Gurobi
#m = Model(solver=GurobiSolver())
using Cbc
m = Model(solver=CbcSolver(seconds=180))
T = 40
L = 3
C = 2
@defVar(m, startat[op=ops,t=1:T], Bin)
@defVar(m, running[op=ops,t=1:T], Bin)
# Run all operations
for op in ops
    @addConstraint(m, sum{startat[op,t], t=1:T} == 1)
end
# At most one operation can start at each time
for t in 1:T
    @addConstraint(m, sum{startat[op,t], op=ops} <= 1)
end
# For each possible starting time for the dependent operations
for t in 1:T-L
    # For each operation
    for op in ops
        for before_op in op.depends
            @addConstraint(m, startat[op,t] <= 
                sum{startat[before_op,s], s=1:t-L})
        end
    end
end
# Link running to startat
for op in ops
    # For each possible start time
    for t in 1:T-L+1
        # For the L units of time including that start time
        for s in t:t+L-1
            @addConstraint(m, running[op,s] >= startat[op,t])
        end
    end
end
# Can't do more than a certain number of ops at a time
add_ops = filter(op->op.optype==:ADD, ops)
mul_ops = filter(op->op.optype==:MUL, ops)
for t in 1:T
    # Multiplies
    @addConstraint(m, sum{running[op,t],op=mul_ops} <= C)
    # Adds
    @addConstraint(m, sum{running[op,t],op=add_ops} <= C)
end
# Prioritize finish time
@setObjective(m, Min, sum{t*startat[final,t],t=1:T} + 1e-5*sum(running))
# How big is the problem?
@show MathProgBase.numvar(m)
@show MathProgBase.numconstr(m)
# Solve it
@time status = solve(m)
@show status

In [ ]:
# Display it
function pretty_matvec(op)
    print(rpad(op.name,15," "))
    for t in 1:T
        if iround(getValue(startat[op,t])) == 1
            print("S")
        elseif iround(getValue(running[op,t])) == 1
            print("R")
        else
            print("_")
        end
    end
    println()
end
map(pretty_matvec, ops);